In [ ]:
import requests
import pandas as pd
import time

In [ ]:
api = 'RGAPI-86856f45-0ac8-4886-b6bd-2d749068b399'
division = ['I' ,'II' , 'III' ,'IV']
leagues = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']
super_leauges = ['masterleagues', 'grandmasterleagues', 'challengerleagues']
path = '/content/drive/MyDrive/LoL api data/'

In [ ]:
import os
directory_path = path
directory_files = os.listdir(directory_path)
directory_files

for i in leagues + super_leauges:
  #print(path + i)
  if not os.path.exists(path + i):
    print('><> creating file ><>')
    os.makedirs(path + i)

In [ ]:
4000 * len(leagues*4 + super_leauges)

108000

In [ ]:
def summoners(elo, division):
  url='https://la1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{}/{}?page=1&api_key={}'.format(elo, division, api)
  db_1 = pd.read_json(requests.get(url).text)

  time.sleep(4)

  url='https://la1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{}/{}?page=2&api_key={}'.format(elo, division, api)
  db_2 = pd.read_json(requests.get(url).text)
  
  return pd.concat([db_1, db_2], ignore_index=True)

In [ ]:
def get_summoner(summonerId):
  l = []
  error = []

  for i in summonerId:
    req= requests.get('https://la1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}'.format(i, api))
    #print(req.status_code)

    if req.status_code == 200:
      l.append(pd.DataFrame(pd.read_json(req.text , typ='series')).T)
    else:
      time.sleep(4)
      error.append(i)
      #print(req.status_code)
  if len(l) != 0:
    return pd.concat(l, ignore_index=True), error
  else:
    return l, error

In [ ]:
def get_summoner_high_elo(division):
  url='https://la1.api.riotgames.com/lol/league/v4/{}/by-queue/RANKED_SOLO_5x5?api_key={}'.format(division, api)
  db = pd.DataFrame(pd.read_json(requests.get(url).text , typ='series')).T
  db_ =pd.DataFrame(db['entries'][0])
  db_['tier'] = db['tier'][0]
  db_['leagueId'] = db['leagueId'][0]
  db_['queue'] = db['queue'][0]
  db_['name'] = db['name'][0]
  print(url)
  return db_

In [ ]:
def get_matchId(puuid, count):
  l = []
  error = []
  #print(url)
  for i in puuid:
    url='https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?type=ranked&start=0&count={}&api_key={}'.format(i, count,api )
    #print(url)
    req= requests.get(url)
    #print(req.status_code)
    
    if req.status_code == 200:
      #l.append(pd.DataFrame(pd.read_json(req.text , typ='series')).T)
      l.append(pd.read_json(req.text))
    else:
      time.sleep(40)
      error.append(i)
      #print(req.status_code)
  if len(l) != 0:
    return pd.concat(l, ignore_index=True), error
  else:
    return l, error


In [ ]:
def get_data_match(requ ,matchId):
  #url = 'https://americas.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(matchId, api)
  #print(url)
  try:
    df_a = pd.read_json(requ.text)

    bf_base = df_a['info']['teams']
    #info
    df_info = []
    #bans
    df_bans = []
    #objt
    df_objt = []

    for i in [0,1]:
      df_info.append(pd.DataFrame([{'win' : pd.DataFrame(bf_base)['win'][i], 'teamId' :  pd.DataFrame(bf_base)['teamId'][i]}]))

      _df_bans = pd.DataFrame(pd.DataFrame(bf_base)['bans'][i])
      _df_bans['teamId'] = pd.DataFrame(bf_base)['teamId'][i]
      df_bans.append(_df_bans)

      _df_objt = pd.DataFrame(pd.DataFrame(bf_base)['objectives'][i])
      d_f = {'baron' : 'baron_first', 'champion' : 'champion_first', 'dragon' : 'dragon_first', 'inhibitor' : 'inhibitor_first', 'riftHerald' : 'riftHerald_first', 'tower' : 'tower_first'}
      d_k = {'baron' : 'baron_kills', 'champion' : 'champion_kills', 'dragon' : 'dragon_kills', 'inhibitor' : 'inhibitor_kills', 'riftHerald' : 'riftHerald_kills', 'tower' : 'tower_kills'}
      f = pd.DataFrame(_df_objt).loc[['first']].rename(columns = d_f).reset_index()
      k = pd.DataFrame(_df_objt).loc[['kills']].rename(columns = d_k).reset_index()
      _df_objt_f  = pd.concat([f, k], axis = 1).drop(['index'], axis= 1)
      _df_objt_f['teamId'] =  pd.DataFrame(bf_base)['teamId'][i]
      df_objt.append(_df_objt_f)

    df_data = df_a.T.drop(['participants', 'teams', 'tournamentCode', 'matchId', ], axis = 1).loc[['info']].reset_index()
    df_data['matchId'] = matchId
    df_data = df_data.drop(['index'], axis= 1)

    df_info = pd.concat(df_info, ignore_index=True)
    df_info['matchId'] = matchId

    df_bans = pd.concat(df_bans, ignore_index=True)
    df_bans['matchId'] = matchId

    df_objt = pd.concat(df_objt, ignore_index=True)
    df_objt['matchId'] = matchId

    df_game= pd.DataFrame(df_a['info']['participants'])
    df_game['matchId'] = matchId

    return df_data, df_info, df_bans, df_objt, df_game
  except:
    print('error here ....', matchId)

In [ ]:
def iter_data_match(matchIds):
  #data
  df_data = []
  #info
  df_info = []
  #bans
  df_bans = []
  #objt
  df_objt = []
  #game
  df_game = []

  #error
  error = []


  for matchId in matchIds:
    url= 'https://americas.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(matchId, api)
    #print(url)

    requ = requests.get(url)


    if requ.status_code == 200:
      try:
        data, info, bans, objt, game = get_data_match(requ, matchId)

        df_data.append(data)
        df_info.append(info)
        df_bans.append(bans)
        df_objt.append(objt)
        df_game.append(game)
      except:
        print('error here ....', matchId)
    else :
      time.sleep(40)
      error.append(matchId)

  if len(df_data) !=0 :
    df_data = pd.concat(df_data, ignore_index=True)
    df_info = pd.concat(df_info, ignore_index=True)
    df_bans = pd.concat(df_bans, ignore_index=True)
    df_objt = pd.concat(df_objt, ignore_index=True)
    df_game = pd.concat(df_game, ignore_index=True)

  return df_data, df_info, df_bans, df_objt, df_game, error

## GET PUUID LOW ELO :C




In [ ]:
for elo in leagues:
  
  da_f = []
  time.sleep(4)

  for d in division:
    print(elo, d)
    da_f.append(summoners(elo, d))
  dLoL_summ = pd.concat(da_f, ignore_index=True)
  
  print('get: ', (len(dLoL_summ) ))

  dLoL_summ.to_csv(path+ elo + '/summ_{}.csv'.format(elo),  index=False)

  
  #dLoL_puid.to_csv('/content/drive/MyDrive/LoL Data/puuid_{}.csv'.format(elo), index_col = False)
  

IRON I
IRON II
IRON III
IRON IV
get:  1640
BRONZE I
BRONZE II
BRONZE III
BRONZE IV
get:  1640
SILVER I
SILVER II
SILVER III
SILVER IV
get:  1640
GOLD I
GOLD II
GOLD III
GOLD IV
get:  1640
PLATINUM I
PLATINUM II
PLATINUM III
PLATINUM IV
get:  1640
DIAMOND I
DIAMOND II
DIAMOND III
DIAMOND IV
get:  1640


In [ ]:
for elo in leagues:
  dLoL_summ = pd.read_csv(path+ elo + '/summ_{}.csv'.format(elo))
  print(elo)
  print(((dLoL_summ['wins'] + dLoL_summ['losses'] > 20).value_counts()))

  dLoL_pud0, error = get_summoner(dLoL_summ['summonerId'])

  l_axd = []
  l_axd.append(dLoL_pud0)

  while len(error) != 0:
    print('erros...: ', len(error))
    time.sleep(40)
    _dLoL_puud, error = get_summoner(error)
    if(len(_dLoL_puud)!=0):
      l_axd.append(_dLoL_puud)

  dLoL_puid = pd.concat(l_axd, ignore_index=True)
  print('size....', len(dLoL_puid))

  
  dLoL_puid.to_csv(path+ elo + '/puid_{}.csv'.format(elo),  index=False)


IRON
True     1261
False     379
dtype: int64
erros...:  238
erros...:  25
size.... 1640
BRONZE
True     1118
False     522
dtype: int64
erros...:  229
erros...:  25
size.... 1640
SILVER
True     1291
False     349
dtype: int64
erros...:  228
erros...:  24
erros...:  8
size.... 1640
GOLD
True     1404
False     236
dtype: int64
erros...:  229
erros...:  25
size.... 1640
PLATINUM
True     1464
False     176
dtype: int64
erros...:  229
erros...:  25
size.... 1640
DIAMOND
True     1571
False      69
dtype: int64
erros...:  240
erros...:  34
size.... 1640


## GET PUUID HIGH ELO :)


In [ ]:
for s_elo in super_leauges:
  dLoL_summ = get_summoner_high_elo(s_elo)
  dLoL_summ.to_csv( path + s_elo + '/summ_{}.csv'.format(s_elo),  index=False)


https://la1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=RGAPI-21902b76-766b-485c-b32b-caec65ca7bb0
https://la1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=RGAPI-21902b76-766b-485c-b32b-caec65ca7bb0
https://la1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=RGAPI-21902b76-766b-485c-b32b-caec65ca7bb0


In [ ]:
for s_elo in reversed(super_leauges):
  dLoL_summ = pd.read_csv(path+ s_elo + '/summ_{}.csv'.format(s_elo))
  print(s_elo, '><<>', len(dLoL_summ))
  print(((dLoL_summ['wins'] + dLoL_summ['losses'] > 20).value_counts()))

  dLoL_pud0, error = get_summoner(dLoL_summ['summonerId'])

  l_axd = []
  l_axd.append(dLoL_pud0)

  while len(error) != 0:
    print('erros...: ', len(error))
    time.sleep(40)
    _dLoL_puud, error = get_summoner(error)
    if(len(_dLoL_puud)!=0):
      l_axd.append(_dLoL_puud)

  dLoL_puid = pd.concat(l_axd, ignore_index=True)

  dLoL_puid.to_csv(path + s_elo+ '/puid_{}.csv'.format(s_elo),  index=False)

challengerleagues ><<> 200
True    200
dtype: int64
erros...:  100
erros...:  36
grandmasterleagues ><<> 500
True    500
dtype: int64
erros...:  302
erros...:  200
erros...:  100
erros...:  94
masterleagues ><<> 2538
True     2532
False       6
dtype: int64
erros...:  1638
erros...:  1017
erros...:  638
erros...:  338
erros...:  238
erros...:  138
erros...:  38
erros...:  38
erros...:  38


## Get matchId

In [ ]:
for s_elo in reversed(super_leauges):
  db_matchs = pd.read_csv(path + s_elo+ '/puid_{}.csv'.format(s_elo))
  count = 20
  if len(db_matchs) < 800:
    count = 80
  dLoL_pud0, error = get_matchId(db_matchs['puuid'], count)

  l_axd = []
  l_axd.append(dLoL_pud0)

  while len(error) != 0:
    print('erros...: ', len(error))
    time.sleep(40)
    _dLoL_puud, error = get_matchId(error, count)
    if(len(_dLoL_puud)!=0):
      l_axd.append(_dLoL_puud)

  dLoL_match = pd.concat(l_axd, ignore_index=True)
  print('size....', len(dLoL_match))

  dLoL_match.to_csv(path + s_elo+ '/match_{}.csv'.format(s_elo),  index=False)

erros...:  3
size.... 16000
erros...:  9
size.... 39979
erros...:  49
size.... 50760


In [ ]:
for s_elo in reversed(leagues):
  db_matchs = pd.read_csv(path + s_elo+ '/puid_{}.csv'.format(s_elo))
  dLoL_pud0, error = get_matchId(db_matchs['puuid'], 20)

  l_axd = []
  l_axd.append(dLoL_pud0)

  while len(error) != 0:
    print('erros...: ', len(error))
    time.sleep(40)
    _dLoL_puud, error = get_matchId(error, count)
    if(len(_dLoL_puud)!=0):
      l_axd.append(_dLoL_puud)

  dLoL_match = pd.concat(l_axd, ignore_index=True)
  print('size....', len(dLoL_match))

  dLoL_match.to_csv(path + s_elo+ '/match_{}.csv'.format(s_elo),  index=False)

erros...:  32
size.... 32800
erros...:  31
size.... 32800
erros...:  31
erros...:  1
size.... 32788
erros...:  31
size.... 32800
erros...:  31
erros...:  1
size.... 32658
erros...:  32
size.... 32682


## Gama Data

In [ ]:
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
form_1 = '/csv'
form_2 = '/parket'
for i in leagues + super_leauges:
  #print(path + i)
  if not os.path.exists(path + i + form_1):
    print('><> creating file ><>')
    os.makedirs(path + i + form_1)
  if not os.path.exists(path + i + form_2):
    print('><> creating file ><>')
    os.makedirs(path + i + form_2)

In [ ]:
#dLoL_pud0, error = get_matchId(db_matchs['puuid'])
def save_data(db_iter, elo, i):
  df_data, df_info, df_bans, df_objt, df_game, error = iter_data_match(db_iter['0'])

  l_df_data = []
  l_df_info = []
  l_df_bans = []
  l_df_objt = []
  l_df_game = []

  l_df_data.append(df_data)
  l_df_info.append(df_info)
  l_df_bans.append(df_bans)
  l_df_objt.append(df_objt)
  l_df_game.append(df_game)

  if len(df_data) == 0:
    print('kill the code')

  else:
    print('working hard  ¦̵̱ ̵̱ ̵̱ ̵̱ ̵̱(̢ ̡͇̅└͇̅┘͇̅ (▤8כ−◦ ....',len(l_df_data))

  while len(error) != 0:
    print('erros...: ', len(error))
    time.sleep(40)
    #_dLoL_puud, error = get_matchId(error)
    try:
      _df_data, _df_info, _df_bans, _df_objt, _df_game, error = iter_data_match(error)

      if(len(_df_data)!=0):
        l_df_data.append(_df_data)
        l_df_info.append(_df_info)
        l_df_bans.append(_df_bans)
        l_df_objt.append(_df_objt)
        l_df_game.append(_df_game)
    except:

      print('fatal error ... __̴ı̴̴̡̡̡ ̡͌l̡̡̡ ̡͌l̡*̡̡ ̴̡ı̴̴̡ ̡̡͡|̲̲̲͡͡͡ ̲▫̲͡ ̲̲̲͡͡π̲̲͡͡ ̲̲͡▫̲̲͡͡ ̲|̡̡̡ ̡ ̴̡ı̴̡̡ ̡͌l̡̡̡̡.___')
      break

  df_df_data = pd.concat(l_df_data, ignore_index=True)
  df_df_info = pd.concat(l_df_info, ignore_index=True)
  df_df_bans = pd.concat(l_df_bans, ignore_index=True)
  df_df_objt = pd.concat(l_df_objt, ignore_index=True)
  df_df_game = pd.concat(l_df_game, ignore_index=True)

  df_df_data.to_parquet(path + elo + form_2 + '/data_{}_batch_{}.parquet'.format(elo, i) ,  index=False, engine = 'pyarrow')
  df_df_info.to_parquet(path + elo + form_2 + '/info_{}_batch_{}.parquet'.format(elo, i) ,  index=False, engine = 'pyarrow')
  df_df_bans.to_parquet(path + elo + form_2 + '/bans_{}_batch_{}.parquet'.format(elo, i) ,  index=False, engine = 'pyarrow')
  df_df_objt.to_parquet(path + elo + form_2 + '/objt_{}_batch_{}.parquet'.format(elo, i) ,  index=False, engine = 'pyarrow')
  df_df_game.to_parquet(path + elo + form_2 + '/game_{}_batch_{}.parquet'.format(elo, i) ,  index=False, engine = 'pyarrow')

  df_df_data.to_csv(path + elo + form_1 + '/data_{}_batch_{}.csv'.format(elo, i) ,  index=False)
  df_df_info.to_csv(path + elo + form_1 + '/info_{}_batch_{}.csv'.format(elo, i) ,  index=False)
  df_df_bans.to_csv(path + elo + form_1 + '/bans_{}_batch_{}.csv'.format(elo, i) ,  index=False)
  df_df_objt.to_csv(path + elo + form_1 + '/objt_{}_batch_{}.csv'.format(elo, i) ,  index=False)
  df_df_game.to_csv(path + elo + form_1 + '/game_{}_batch_{}.csv'.format(elo, i) ,  index=False)


In [ ]:
for elo in reversed(leagues + super_leauges):
  #read data
  db_match = pd.read_csv(path + elo + '/match_{}.csv'.format(elo))
  print('original....', len(db_match))
  db_match = db_match.drop_duplicates()

  l = int(len(db_match)/4) 
  print(elo, ' ><> ' , len(db_match), ' <>< ', l)
  
  batch_0 = db_match.iloc[   :l*1,:]
  batch_1 = db_match.iloc[l  :l*2,:]
  batch_2 = db_match.iloc[l*2:l*3,:]
  batch_3 = db_match.iloc[l*3:   ,:]
  batch = [batch_0, batch_1, batch_2, batch_3]

  for i in range(0,4):
    if not os.path.exists(path + elo + form_2 + '/data_{}_batch_{}.parquet'.format(elo, i)):
      print(path + elo + form_2 + '/data_{}_batch_{}.parquet'.format(elo, i), '.... not found :c')
      print('...')
      save_data(batch[i], elo, i)
    else:
      print('/data_{}_batch_{}'.format(elo, i), '.... complete')




original.... 16000
challengerleagues  ><>  11930  <><  2982
/data_challengerleagues_batch_0 .... complete
/data_challengerleagues_batch_1 .... complete
/data_challengerleagues_batch_2 .... complete
/data_challengerleagues_batch_3 .... complete
original.... 39979
grandmasterleagues  ><>  23263  <><  5815
/data_grandmasterleagues_batch_0 .... complete
/data_grandmasterleagues_batch_1 .... complete
/data_grandmasterleagues_batch_2 .... complete
/data_grandmasterleagues_batch_3 .... complete
original.... 50760
masterleagues  ><>  27545  <><  6886
/data_masterleagues_batch_0 .... complete
/content/drive/MyDrive/LoL api data/masterleagues/parket/data_masterleagues_batch_1.parquet .... not found :c
...
error here .... LA1_1366451657
error here .... LA1_1366451657
working hard  ¦̵̱ ̵̱ ̵̱ ̵̱ ̵̱(̢ ̡͇̅└͇̅┘͇̅ (▤8כ−◦ .... 1
erros...:  134
/content/drive/MyDrive/LoL api data/masterleagues/parket/data_masterleagues_batch_2.parquet .... not found :c
...


KeyboardInterrupt: ignored